In [493]:
import pandas as pd
import numpy as np
import math

In [494]:
RANDOM_SEED = 13022022

In [495]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [496]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [497]:
test.sample(3)

,Smiles
1212,CC(C)OC(=O)C(C)NP(=O)(COC(C)Cn1cnc2c(N)ncnc21)...
124,COc1ccc2c(c1OCc1ccccc1)C[C@@H](C(=O)O)N(C(=O)C...
1601,Br.Cc1ccc(-c2cn3c4c(sc3n2)CCCC4)cc1


In [498]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [499]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [500]:
train.sample(3)

,Smiles,Active
5175,CN(C)c1ccc(/C=C2\C(=O)Nc3ccccc32)cc1,False
3510,Cc1ccc(C)c(Oc2ccncc2C(=O)N2CCN(C3CC3)c3ccccc32)c1,False
5053,CCOC(=O)c1ccc(-c2ccc(OC(=O)N(C)Cc3cccc(-c4ccnc...,False


In [501]:
list(train['Active'].unique())

[False, True]

In [502]:
list_active = {False: 0, True: 1}

In [503]:
train['Active'] = train['Active'].map(list_active)

In [504]:
train.sample()

,Smiles,Active
3734,CNCC(O)c1ccc(O)cc1,0


In [505]:
train['train'] = 1
test['train'] = 0

In [506]:
df = pd.concat([train, test], ignore_index=True)

In [507]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [508]:
df['train'].unique()

array([1, 0], dtype=int64)

In [509]:
df[df['train'] == 1]['Active'].value_counts()

0.00    5351
1.00     206
Name: Active, dtype: int64

In [510]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [511]:
atoms['C']

67272

In [512]:
new_columnes = list(atoms.keys())

In [513]:
print(len(new_columnes))

45


In [514]:
df['len_smiles'] = df['Smiles'].apply(lambda x: len(x))

In [515]:
df['len_smiles'].max()

707

In [516]:
df['len_smiles'].describe()

count   7171.00
mean      51.12
std       31.88
min        6.00
25%       34.00
50%       46.00
75%       60.00
max      707.00
Name: len_smiles, dtype: float64

In [517]:
df[df['len_smiles'] > 60]['Smiles'].count()

1732

In [518]:
df['smiles_list'] = df['Smiles'].apply(lambda x: list(x))

In [519]:
def count_i(x):
    atoms = Counter()
    for i in x:
        atoms[i] += 1
    return atoms

In [520]:
df['smiles_list'] = df['smiles_list'].apply(lambda x: count_i(x))

In [521]:
for i in new_columnes:
    df[i] = df['smiles_list'].apply(lambda x: x[i])

In [522]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
9,CC(=O)N(O)C/C=C/c1cccc(Oc2ccccc2)c1,0.00,1,35,"{'C': 5, '(': 3, '=': 2, 'O': 3, ')': 3, 'N': ...",5,3,12,2,2,...,0,0,0,0,0,0,0,0,0,0
6829,COc1ccc(C(=O)NCc2cccnc2)cc1C(=O)NCc1cccnc1,nan,0,42,"{'C': 5, 'O': 3, 'c': 16, '1': 4, '(': 3, '=':...",5,3,16,4,2,...,0,0,0,0,0,0,0,0,0,0
1432,O=C(O)c1ccc(N2N=C(Cc3ccccc3)/C(=C\c3ccc(-c4ccc...,0.00,1,62,"{'O': 3, '=': 4, 'C': 6, '(': 5, ')': 5, 'c': ...",6,3,22,2,2,...,0,0,0,0,0,0,0,0,0,0


In [523]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
4088,CC(C)(N)Cc1ccccc1.Cl,0.00,1,20,"{'C': 5, '(': 2, ')': 2, 'N': 1, 'c': 6, '1': ...",5,0,6,2,0,...,0,0,0,0,0,0,0,0,0,0
3163,CN(C)CCCO[C@H]1[C@@H]([C@H](O)CO)O[C@@H]2OC(C)...,0.00,1,60,"{'C': 15, 'N': 1, '(': 5, ')': 5, 'O': 6, '[':...",15,6,0,2,2,...,0,0,0,0,0,0,0,0,0,0
3829,O=C(NC[C@H](O)CN1CCc2ccccc2C1)c1cc(NC2COC2)ncn1,0.00,1,47,"{'O': 3, '=': 1, 'C': 10, '(': 3, 'N': 3, '[':...",10,3,10,4,4,...,0,0,0,0,0,0,0,0,0,0


In [524]:
df.drop(['smiles_list'], axis = 1, inplace = True)

In [525]:
df.set_index(['Smiles'], inplace = True)

In [526]:
df.sample(3)

,Active,train,len_smiles,C,O,c,1,2,[,n,...,7,8,e,A,K,M,g,i,L,9
Smiles,,,,,,,,,,,,,,,,,,,,,
Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,nan,0,38,7,0,12,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
N[C@@H](CCC(=O)N[C@@H](CSN=O)C(=O)NCC(=O)O)C(=O)O,0.00,1,49,10,7,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
Cc1cc(C(=O)O)sc1C(N)C(=O)O,0.00,1,26,4,4,4,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [527]:
#df.to_csv('df_first_draft.csv')

In [528]:
train_new = df[df['train'] == 1]
test_new = df[df['train'] == 0]

In [529]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5557 entries, COc1ccc2[nH]cc(CCN)c2c1 to CCCCNc1ccc(C(=O)OCCN(C)C)cc1.Cl
Data columns (total 48 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Active      5557 non-null   float64
 1   train       5557 non-null   int64  
 2   len_smiles  5557 non-null   int64  
 3   C           5557 non-null   int64  
 4   O           5557 non-null   int64  
 5   c           5557 non-null   int64  
 6   1           5557 non-null   int64  
 7   2           5557 non-null   int64  
 8   [           5557 non-null   int64  
 9   n           5557 non-null   int64  
 10  H           5557 non-null   int64  
 11  ]           5557 non-null   int64  
 12  (           5557 non-null   int64  
 13  N           5557 non-null   int64  
 14  )           5557 non-null   int64  
 15  @           5557 non-null   int64  
 16  .           5557 non-null   int64  
 17  l           5557 non-null   int64  
 18  =           5557 non-nul

In [530]:
X = train_new.drop(['Active'], axis = 1)
y = train_new['Active']

In [531]:
from sklearn import model_selection, datasets, metrics, tree 

In [532]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size=0.30, random_state=42)

In [533]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [534]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_labels)

DecisionTreeClassifier()

In [535]:
predictions_clf = clf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf)

0.10218978102189781

In [536]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_labels)

DecisionTreeClassifier()

In [537]:
predictions_clf = clf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf)

0.15942028985507245

In [538]:
test_new_data = test_new.drop(['Active'], axis = 1) 

In [539]:
predictions = clf.predict(test_new_data)

In [540]:
test_new_data['Active'] = predictions

In [541]:
test_new_data.sample(5)

,train,len_smiles,C,O,c,1,2,[,n,H,...,8,e,A,K,M,g,i,L,9,Active
Smiles,,,,,,,,,,,,,,,,,,,,,
Cc1nc(N)sc1CCN,0,14,3,0,3,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.00
CCCc1c(OCCCCc2nnn[nH]2)ccc(C(C)=O)c1O,0,37,9,3,7,2,2,1,4,1,...,0,0,0,0,0,0,0,0,0,0.00
CC(C)(C)c1cc(C(C)(C)C)c2c(c1)C(O)(C(F)(F)F)C(=O)O2,0,50,11,3,6,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.00
C=C(c1ccccc1OCc1cccc(Cl)c1)n1ccnc1,0,34,4,1,15,6,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0.00
C[S+](CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@H](O)[C@@H]1O,0,71,10,5,5,2,2,6,4,5,...,0,0,0,0,0,0,0,0,0,1.00


In [542]:
test_new_data.reset_index(inplace = True)

In [543]:
test_new_data.sample(5)

,Smiles,train,len_smiles,C,O,c,1,2,[,n,...,8,e,A,K,M,g,i,L,9,Active
787,N[C@@H](CC(=O)N1CCn2c(nnc2C(F)(F)F)C1)Cc1cc(F)...,0,54,8,1,8,4,2,1,3,...,0,0,0,0,0,0,0,0,0,0.00
949,COc1cc(COc2cc(N)c(Cl)cc2C(=O)CCC2CCN(CCNS(C)(=...,0,63,15,6,12,2,4,0,0,...,0,0,0,0,0,0,0,0,0,0.00
806,CNC(=O)c1ccc(-c2cc(Cl)cc(Cl)c2Cl)c(N)n1,0,39,5,1,11,2,2,0,1,...,0,0,0,0,0,0,0,0,0,0.00
80,N#Cc1c(NS(=O)(=O)c2ccccc2)n(Cc2ccccc2)c2nc3ccc...,0,53,2,2,22,2,6,0,3,...,0,0,0,0,0,0,0,0,0,0.00
86,COc1ccc(Cn2c(CCc3ccccc3)nnc2[C@@H](Cc2c[nH]c3c...,0,65,8,2,22,2,4,2,4,...,0,0,0,0,0,0,0,0,0,0.00


In [544]:
submission = test_new_data[['Smiles', 'Active']]

In [545]:
submission.sample(4)

,Smiles,Active
397,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)OCOC(=O)C(C...,0.00
222,CC(C)CN(C[C@@H](OP(=O)(O)O)[C@H](Cc1ccccc1)NC(...,0.00
282,CCCCCCCCCCCCOP(=O)(O)O,0.00
1164,CN1CCN2c3ccccc3Cc3ccccc3C2C1.Cl,0.00


In [546]:
submission['Active'] = submission['Active'].apply(lambda x: int(x))

In [547]:
submission['Active'].value_counts()

0    1537
1      77
Name: Active, dtype: int64

In [548]:
submission.to_csv('submission.csv', index=False)